Set up Cross-Validation for this

In [37]:
!pip install pyspark

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
import matplotlib.pyplot as plt
import numpy as np
# import re
# import random
import pandas as pd
# import seaborn as sns
# from collections import Counter, defaultdict
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve, auc, roc_auc_score
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import lit, to_date
from pyspark.sql.functions import concat_ws,col, lpad
import json
file_path_to_json
files = json.load(open(file_path_to_json,'r'))
pd.options.display.max_rows = 100
pd.options.display.width = 150
RANDOM_SEED = 696


In [40]:
dfw_train = pd.read_parquet(files['Final_Sets']['Training']['Train_All'])

In [41]:
dfw_train.shape

(293218, 116)

Double check for nan

In [42]:
dfw_train.columns[dfw_train.isna().any()].tolist()

[]

Create the Train and Dev dataframes

In [43]:
# # train_df = dfw_train[dfw_train['Year'] < 2019].copy()
# # dev_df = dfw_train[dfw_train['Year'] == 2019].copy()
# dfw_label1 = dfw_train['Weather_Label'].copy()
# train_df, dev_df, y_train1, y_dev1 = train_test_split(dfw_train, dfw_label1, test_size=0.2, random_state=42)
# print(train_df.shape)
# print(dev_df.shape)

In [44]:
# check = train_df.shape[0] + dev_df.shape[0]
# check

In [45]:
def get_downsampled_set(train,perc_pos_class):
  # Function obtains a downsampled training set, has no effect on validation
  X_train = train.copy()
  #Split the pos and neg classes. Obtain fraction of neg classes per downsample
  X_train_pos = X_train[X_train['Weather_Label']==1]
  # Determine # of neg samples and randomly sample that amount
  neg_samples = int(len(X_train_pos)/perc_pos_class)
  print("len(xtrain pos)", len(X_train_pos))
  print("neg_samples", neg_samples)
  X_train_neg = X_train[X_train['Weather_Label']==0].sample(neg_samples)
  #Combine the pos and neg samples and shuffle them
  X_train_sample = pd.concat([X_train_pos, X_train_neg])
  X_train_sample = X_train_sample.sample(frac=1)
  # Assign y's
  y_train_sample = X_train_sample['Weather_Label']
  # Remove the target from X datasets
  X_train_sample = X_train_sample.iloc[:,:-1]
  # Sample upweighting
  sample_weights = np.ones(len(y_train_sample))
  sample_weights[y_train_sample == 1] = len(y_train_sample) / sum(y_train_sample == 1)
  return X_train_sample, y_train_sample, sample_weights

In [47]:
X_train, y_train, weights = get_downsampled_set(dfw_train, .2)

len(xtrain pos) 4352
neg_samples 21760


In [48]:
print(X_train.shape)
# print(X_dev.shape)
print(y_train.shape)
# print(y_dev.shape)
print(weights.shape)

(26112, 115)
(26112,)
(26112,)


In [49]:
type(weights)

numpy.ndarray

In [50]:
X_train_complete = pd.concat([X_train, y_train], axis = 1)
X_train_complete.shape

(26112, 116)

In [51]:
X_train_complete['weights'] = weights.tolist()
X_train_complete.shape

(26112, 117)

In [52]:
X_Dev_complete = pd.concat([X_dev, y_dev], axis = 1)
X_Dev_complete.shape

(58644, 116)

In [53]:
X_train_complete.to_parquet(files['Final_Sets']['Training']['Train_Down'])
X_Dev_complete.to_parquet(files['Final_Sets']['Training']['Dev_Down'])